# BERT Transfer Learning for Sentiment Analysis
**Objective:** Provide a framework to perform transfer learning using the pre-trained distilBERT model, 
allowing options for fine-tuning the distilBERT model or simply use its outputs as features. In this example, 
we use a dataset of Yelp reviews and build a sentiment classifier to identify whether a 
review is 1 or 5 stars.

In [ ]:
import pandas as pd
import torch
import transformers
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertModel, DistilBertTokenizer
import torch
import numpy as np
import json
from tqdm import tqdm
device = 'cpu'
device = 'gpu'

# Data pre-processing
**Summary:** parse Yelp reviews for the review text and number of stars associated with that review.
Only parse reviews with low or high stars, and ensure that we have an equal number of low and high star reviews. Low star reviews have 1 or 2 stars, and high star reviews have 5 stars.<br /> 
- *lowstar_review_limit*: once we parse this number of low star reviews have been parsed. Break from processing. Typically there are more high star than low star reviews so the total number of reviews read in will be twice this number. <br /> 
- *review_limit*: once a total of this number of reviews have been parsed, stop reading in more.<br /> 
- *sample_per_cat*: sample this many low star and high star reviews respectively from what is parsed.<br /> 
- *max_num_words*: only parse reviews with number of words less than this length <br /> 
[Download the data](https://www.yelp.com/dataset/documentation/main) <br /> 

In [3]:
#read-in data
lowstar_review_limit = 1024
review_limit = np.inf
sample_per_cat = 1024
max_num_words = 50


path = "/home/pd/datasets/yelp_reviews/yelp_reviews.json"
review_fields_wanted = ['text','lowstar']
rev = pd.DataFrame(columns=review_fields_wanted)
with open(path,encoding='utf-8') as d:
    counter = 0
    lowstar_counter = 0
    for line in d:
        L = json.loads(line)
        lowstar = L['stars'] == 1 or (L['stars'] == 2)
        fivestar = L['stars'] == 5
        not1or5 = not(lowstar or fivestar)
        if len(L['text'].split()) > max_num_words or not1or5:
            continue
        if lowstar:
            lowstar_counter += 1
            L['lowstar'] = 1
        else:
            L['lowstar'] = 0
        less_fields = {key: L[key] for key in review_fields_wanted }
        rev.loc[counter] = less_fields
        counter += 1
        if counter == review_limit or lowstar_counter == lowstar_review_limit:
            break

            
rev = rev.rename(columns = {'text':'_text','lowstar':'_lowstar'})


rev = rev.groupby('_lowstar').apply(lambda x: x.sample(sample_per_cat)).reset_index(drop=True)
rev['TARGETS'] = rev['_lowstar']
print(f'Number of 1 star reviews:{rev._lowstar[rev._lowstar == 1].count()}')
print(f'Number of 5 star reviews:{rev._lowstar[rev._lowstar == 0].count()}')
print(rev._text[rev._lowstar == 1].sample(5))
print(rev._text[rev._lowstar == 0].sample(5))


Number of 1 star reviews:1024
Number of 5 star reviews:1024
2047    I don't understand why this place has a line o...
1088    I got the gyro "platter" for lunch, which was ...
1255    Had great reviews. Tried someone new..my husba...
1843    Breakfast - burnt pork sausage, very dark almo...
1870    I really, really don't like Karaoke.   It's re...
Name: _text, dtype: object
392    A great place for cheap, ripe produce. Because...
966    Actually gabbed a cheesesteak from next door a...
376    Hannah is wonderful! I have come to her severa...
951    Quaint place for a glass of wine while people ...
53     Fantastic food great environment to just shop,...
Name: _text, dtype: object


In [5]:
rev.to_csv('/home/pd/datasets/yelp_reviews/yelp_review_2048.csv')

In [29]:
#def dataset
class DFToTokenized(Dataset):
    def __init__(self,df,tokenizer,max_len):
        self.len = len(df)
        self.data = df
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self,index):

        review = ' '.join(self.data['_text'][index].split())
        inp = self.tokenizer.encode_plus(
            review,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        tokens = inp['input_ids']
        mask = inp['attention_mask']

        return {
            'ids': torch.tensor(tokens, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'targets': torch.tensor(self.data.TARGETS[index], dtype=torch.uint8)
        } 

    def __len__(self):
        return self.len
        



In [42]:
#init train/test params
MAX_LEN = 64
TRAIN_BATCH_SIZE = 64
EPOCHS = 10
LEARNING_RATE = 1e-05
AUTO_SCALE_GRAD = False
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased')

train_frac = 0.8
train_dataset=rev.sample(frac=train_frac,random_state=200)
test_dataset=rev.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

training_set = DFToTokenized(train_dataset, tokenizer, MAX_LEN)
testing_set = DFToTokenized(test_dataset, tokenizer, MAX_LEN)

train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

single_params = {'batch_size': 1,
                'shuffle': True,
                'num_workers': 0
}

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

single_loader = DataLoader(training_set,**single_params)

In [31]:
#def model
class DBertMultiCat(torch.nn.Module):
    def __init__(self):
        super(DBertMultiCat, self).__init__()
        self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 2)

    def forward(self, input_ids, attention_mask):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [32]:
#INIT model, loss, optimizer
model = DBertMultiCat()
for p in model.l1.parameters():
    p.requires_grad = False
model.to(device)
loss_function = torch.nn.CrossEntropyLoss()
params_with_grad = filter(lambda p: p.requires_grad, model.parameters())
optimizer = torch.optim.Adam(params =  params_with_grad, lr=LEARNING_RATE)
if AUTO_SCALE_GRAD:
    scaler = torch.cuda.amp.GradScaler()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [33]:
#train loop def
def calcuate_accu(big_idx, targets):
    n_correct = (big_idx==targets).sum().item()
    return n_correct

def train(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model.train()
    for _,data in tqdm(enumerate(training_loader, 0),total=len(training_loader),
        position=0, leave=True):

        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.uint8)

        if AUTO_SCALE_GRAD:
            with torch.cpu.amp.autocast():
                outputs = model(ids, mask)
                loss = loss_function(outputs, targets)
        else:
            outputs = model(ids, mask)
            loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calcuate_accu(big_idx, targets)

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)
        
        if _%5000==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples 
            print(f"Training Loss per 5000 steps: {loss_step}")
            print(f"Training Accuracy per 5000 steps: {accu_step}")


        optimizer.zero_grad(set_to_none=True)
        if(AUTO_SCALE_GRAD):
            scaler.scale(loss).backward()
            # # When using GPU
            scaler.step(optimizer)
            scaler.update()
        else:
            loss.backward()
            optimizer.step()

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    return 

In [35]:
#train engine
for epoch in range(30):
    train(epoch)

  4%|▍         | 1/26 [00:02<00:55,  2.21s/it]

Training Loss per 5000 steps: 0.687899649143219
Training Accuracy per 5000 steps: 62.5


100%|██████████| 26/26 [01:02<00:00,  2.40s/it]


The Total Accuracy for Epoch 0: 52.747252747252745
Training Loss Epoch: 0.6927502017754775
Training Accuracy Epoch: 52.747252747252745


  4%|▍         | 1/26 [00:02<01:06,  2.67s/it]

...

The Total Accuracy for Epoch 28: 62.27106227106227
Training Loss Epoch: 0.6637566593977121
Training Accuracy Epoch: 62.27106227106227


  4%|▍         | 1/26 [00:23<09:54, 23.76s/it]

Training Loss per 5000 steps: 0.7042104601860046
Training Accuracy per 5000 steps: 50.0


100%|██████████| 26/26 [07:04<00:00, 16.34s/it]

The Total Accuracy for Epoch 29: 59.70695970695971
Training Loss Epoch: 0.666911624945127
Training Accuracy Epoch: 59.70695970695971


In [36]:
for epoch in range(30):
    train(epoch)

  4%|▍         | 1/26 [00:15<06:17, 15.09s/it]

Training Loss per 5000 steps: 0.6586676239967346
Training Accuracy per 5000 steps: 64.0625


100%|██████████| 26/26 [06:10<00:00, 14.26s/it]


The Total Accuracy for Epoch 0: 61.35531135531136
Training Loss Epoch: 0.6642708870080801
Training Accuracy Epoch: 61.35531135531136


  4%|▍         | 1/26 [00:14<05:55, 14.23s/it]

...

The Total Accuracy for Epoch 28: 63.614163614163616
Training Loss Epoch: 0.6458763411411872
Training Accuracy Epoch: 63.614163614163616


  4%|▍         | 1/26 [00:02<01:10,  2.83s/it]

Training Loss per 5000 steps: 0.6600029468536377
Training Accuracy per 5000 steps: 64.0625


100%|██████████| 26/26 [01:26<00:00,  3.32s/it]

The Total Accuracy for Epoch 29: 63.98046398046398
Training Loss Epoch: 0.6436567489917462
Training Accuracy Epoch: 63.98046398046398


In [ ]:
def summarize_results()

In [51]:
#validation def
VALID_BATCH_SIZE = 1 #set to 1 for printing of individual wrong predictions

#detokenize def
def DBDetokenize(a):
    a_orig = [tokenizer.decode(x) for x in a['ids'].squeeze().tolist() if x != 0]
    a_orig = ([x.replace(' ' , '') for x in a_orig])
    return " ".join(a_orig)

def valid(model, testing_loader):
    tr_loss = 0 #added
    nb_tr_steps = 0 #added
    nb_tr_examples = 0 #added
    max_wrong_outputs = 10
    wrong_outputs = 0
    model.eval()
    n_correct = 0; n_wrong = 0; total = 0
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.long)
            outputs = model(ids, mask)#.squeeze()
            loss = loss_function(outputs, targets)
            tr_loss += loss.item()
            big_val, big_idx = torch.max(outputs.data, dim=1)
            n_correct += calcuate_accu(big_idx, targets)

            #print individual wrong responses to file
            if VALID_BATCH_SIZE == 1 and wrong_outputs < max_wrong_outputs: 
                wrong_outputs += 1
                path = '/home/pd/summar   ies/yelp_summary_13Mar23.txt'
                with open(path,'a') as f:
                    f.write(DBDetokenize(data))
                    f.write(f'Should be: {1 if targets.item() else 5}')
                    f.write('\n')
            nb_tr_steps += 1
            nb_tr_examples+=targets.size(0)
            
            if _%5000==0:
                loss_step = tr_loss/nb_tr_steps
                accu_step = (n_correct*100)/nb_tr_examples
                print(f"Validation Loss per 100 steps: {loss_step}")
                print(f"Validation Accuracy per 100 steps: {accu_step}")
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Validation Loss Epoch: {epoch_loss}")
    print(f"Validation Accuracy Epoch: {epoch_accu}")
    
    return epoch_accu

In [52]:
#validation run
acc = valid(model, testing_loader)
print("Accuracy on test data = %0.2f%%" % acc)

/home/pd/NNBasics/venv_NNBasics/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Validation Loss per 100 steps: 0.4546707570552826
Validation Accuracy per 100 steps: 100.0
Validation Loss Epoch: 0.6451624179395233
Validation Accuracy Epoch: 64.8780487804878
Accuracy on test data = 64.88%


In [37]:
output_model_file = '/home/pd/models/yelp_sentiment.bin'
output_vocab_file = '/home/pd/models/yelp_sentiment_vocab.bin'

model_to_save = model
torch.save(model_to_save, output_model_file)
tokenizer.save_vocabulary(output_vocab_file)

('/home/pd/models/yelp_sentiment_vocab.bin',)

In [19]:
#get raw dbert
dbase = DistilBertModel.from_pretrained("distilbert-base-uncased")
a = next(iter(single_loader))
def get_raw_dbert(a, dbase): 
    return dbase(input_ids=a['ids'],attention_mask=a['mask'])

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/home/pd/NNBasics/venv_NNBasics/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the lon

In [30]:
exec(open('/home/pd/NNBasics/NNBasics/src/PostBertEngine.py').read())

2023-03-22 10:16:50.800780: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-22 10:16:52.413475: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


KeyboardInterrupt: 

In [18]:
a = torch.load('/home/pd/datasets/yelp_reviews/yelp_reviews_targets_1024.pt')
a.shape

torch.Size([1638, 1])

In [69]:
exec(open('/home/pd/NNBasics/NNBasics/src/Preprocess.py').read())

In [70]:
DATA_CSV_PATH = "/home/pd/datasets/yelp_reviews_temp/yelp_reviews_2048.csv"
DATA_PATH = "/home/pd/datasets/yelp_reviews/yelp_reviews.json"
SUMMARY_PATH = '/home/pd/summaries/yelp_summary_13Mar23.txt'
OUTPUT_MODEL_PATH = '/home/pd/models/yelp_sentiment.bin'
OUTPUT_VOCAB_PATH = '/home/pd/models/yelp_sentiment_vocab.bin'
BERT_OUT_PATH = '/home/pd/datasets/yelp_reviews_temp/yelp_reviews_postBERT_1024.pt'
TARG_OUT_PATH = '/home/pd/datasets/yelp_reviews_temp/yelp_reviews_targets_1024.pt'

In [74]:
paramd = dict()
paramd['data_path'] = DATA_PATH
paramd['out_path'] = "/home/pd/datasets/yelp_reviews_temp/yelp_reviews_2048.csv"
paramd['max_words'] = 64
paramd['low_star_rev_lim'] = 100
paramd['tot_rev_lim'] = float('inf')
paramd['debug_path'] = '/home/pd/summaries/yelp_temp.txt'

ypp = YelpPreprocess(**paramd)

In [75]:
ypp.data_path

'/home/pd/datasets/yelp_reviews/yelp_reviews.json'

In [76]:
rev = ypp.json_to_csv()

In [1]:
class model():
    stat = 50

    def __init__(self,inp):
        self.inp = inp

    def square(self):
        self.inp = self.inp**2

    @staticmethod
    def buzz(a,b):
        return a + b
    
    def add(self):
        self.inp = self.buzz(self.inp,self.inp)

    def addstat(self):
        self.inp = self.inp + self.stat

class foo():

    def __init__(self,inp):
        self.inp = inp

    def square_obj(self):
        self.inp.square()



In [2]:
a = model(5)
a.addstat()
a.inp

55

In [3]:
DATA_CSV_PATH = "/home/pd/datasets/yelp_reviews/yelp_reviews_2048.csv"
DATA_PATH = "/home/pd/datasets/yelp_reviews/yelp_reviews.json"
SUMMARY_PATH = '/home/pd/summaries/yelp_summary_24Mar23.txt'
OUTPUT_MODEL_PATH = '/home/pd/models/yelp_sentiment.bin'
OUTPUT_VOCAB_PATH = '/home/pd/models/yelp_sentiment_vocab.bin'
BERT_OUT_PATH = '/home/pd/datasets/yelp_reviews/yelp_reviews_postBERT_1024.pt'
TARG_OUT_PATH = '/home/pd/datasets/yelp_reviews/yelp_reviews_targets_1024.pt'


SAVE_BERT_LAYER_OUT = False
FROM_BERT_LAYER_OUT = not SAVE_BERT_LAYER_OUT

FROM_CSV_CONDENSED = True #otherwise getting from full json file

#read-in data
lowstar_review_limit = 128
review_limit = float('inf')
sample_per_cat = 128
max_num_words = 50

#init train/test params
MAX_LEN = 64
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 1 #set to 1 for printing of individual wrong predictions
EPOCHS = 5
LEARNING_RATE = 1e-05
AUTO_SCALE_GRAD = False

In [6]:
from Model import DBertMultiCat

ModuleNotFoundError: No module named 'Model'

In [4]:
a = None
if not a:
    print('hi')
else:
    print('you')

hi
